In [ ]:
# Importar las librerías necesarias
import requests
from bs4 import BeautifulSoup
import datetime as dt
from datetime import timedelta
import numpy as np
import pandas as pd
# Definir la URL de la página web a scrapear
url = "https://www.fctables.com/todays-match-predictions/"

# Obtener el contenido de la página web
response = requests.get(url)

# Crear un objeto BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Buscar los elementos de la página web que queremos extraer
elementos = soup.find_all(class_="table table-striped table-bordered stage-table table-condensed")

df = pd.DataFrame({"Matches":[],
                    "Corners for per game local":[],
                    "Corners for per game visitante":[],
                    "Corners for per game total":[]})

if elementos[0]:
  tbody = elementos[0].find("tbody")
  trs =tbody.find_all("tr")
  for tr in trs:
    url_match = tr.find_all("td")[2].find("a")["href"]
    matchh = tr.find_all("td")[2].find("a").text
    #print(url_match)
    response = requests.get("https://www.fctables.com"+url_match)
    soup = BeautifulSoup(response.content, "html.parser")
    tablas = soup.find_all(id="team_stats_vs")

    for tabla in tablas:
      tbody = tabla.find("tbody")
      trs =tbody.find_all("tr")
      for tr in trs:
        #print(tr)
        if "Corners for per game" in tr.text:
          tds = tr.find_all("td")
          if tds[0].text:
            local = float(tds[0].text)
          else:
            local = 0
          if tds[2].text:
            visitante = float(tds[2].text)
          else:
            visitante = 0
          if local + visitante >= 12:
            df = df.append( pd.DataFrame({"Matches":[matchh],
                    "Corners for per game local":[local],
                    "Corners for per game visitante":[visitante],
                    "Corners for per game total":[local+visitante]}) )
            dic = {'encuentro':matchh,'local':local,'visitante':visitante , 'total': local+visitante }
print(df)
current_date = dt.date.today()
today = current_date.strftime("%w_%B_%Y")
df.to_excel("corners_"+today+".xlsx")

<ipython-input-1-de1d20d884fc>:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append( pd.DataFrame({"Matches":[matchh],


                        Matches  Corners for per game local  \
0  Seattle.. 0-1 Los Angeles FC                        6.03   

   Corners for per game visitante  Corners for per game total  
0                            6.08                       12.11  


In [ ]:
df

,Matches,Corners for per game local,Corners for per game visitante,Corners for per game total
